# Import Library

In [1]:
import os
import csv
import pandas as pd
import shutil
#os.chdir('/mnt/c/Workspace/Ch3/wflow/')
#os.chdir('/mnt/d/')
os.chdir(os.path.join("D:\\GitHub\\wflow\\notebooks"))
import TempBuilder # importing custom built functions.

In [2]:
# Change back working directory to D
os.chdir(os.path.join("D:\\"))
os.getcwd()

'D:\\'

# Copy the instate/staticmaps corresponding to the model config that is to be used

In [3]:
wsName_list = ["san-diego_california_20331196_11023340",
               "fulton_georgia_2047963_02336410"
              ]

#wsName_list = ["fulton_georgia_2047963_02336410"]
wsName_list = ["san-diego_california_20331196_11023340"]
config = str("90") #"90" or "1km"

In [4]:
#import logging
from datetime import datetime

#logging.basicConfig(filename='copy_log.log', level=logging.INFO, format='%(asctime)s - %(message)s')
def copytree_and_overwrite(src, dst):
    if os.path.exists(dst):
        shutil.rmtree(dst)
    shutil.copytree(src, dst)

for n in wsName_list:
    print(str("Processing static/instate: " + n + " in " + config))
    static_origin = os.path.join("./wflow_models/", str(n+"/inputs_" + config), "staticmaps")
    instate_origin = os.path.join("./wflow_models/", str(n+"/inputs_" + config), "instate")
    static_dst = os.path.join("./wflow_models/", str(n), "staticmaps")
    instate_dst = os.path.join("./wflow_models/", str(n), "instate")
    
    copytree_and_overwrite(static_origin, static_dst)
    copytree_and_overwrite(instate_origin, instate_dst)
        
    # Write log entry
    log_file_path = os.path.join("./wflow_models/", str(n), "log_instate_static_copy.txt")
    with open(log_file_path, 'a') as log_file:
        log_entry = f'{datetime.now()} - Copied for: {n}, Config: {config}\n'
        log_file.write(log_entry)


Processing static/instate: san-diego_california_20331196_11023340 in 90


# TempBuilder

# Loop

In [6]:
csv_file_path = os.path.join(str("./wflow_models/san-diego_california_20331196_11023340_768.csv")) ##set this correctly!!!

df = pd.read_csv(csv_file_path)
df.set_index('row_number_full', inplace=True)

for index, row in df.iterrows():
    print(f"\r{index} / {len(df)}", end='    ', flush=True)
    print(row.iloc[5])

1 / 768    0.297
2 / 768    0.584
3 / 768    0.048
6 / 768    0.462
7 / 768    0.894
10 / 768    0.193
11 / 768    0.744
13 / 768    0.148
14 / 768    0.795
15 / 768    0.379
16 / 768    0.58
17 / 768    0.173
18 / 768    0.966
19 / 768    0.422
20 / 768    0.702
21 / 768    0.771
22 / 768    0.128
23 / 768    0.536
25 / 768    0.62
26 / 768    0.337
27 / 768    0.866
28 / 768    0.076
29 / 768    0.501
31 / 768    0.271
32 / 768    0.91
33 / 768    0.746
35 / 768    0.981
36 / 768    0.218
37 / 768    0.392
38 / 768    0.552
39 / 768    0.143
41 / 768    0.061
42 / 768    0.82
43 / 768    0.292
44 / 768    0.605
45 / 768    0.896
46 / 768    0.226
47 / 768    0.65
49 / 768    0.864
50 / 768    0.093
51 / 768    0.629
52 / 768    0.312
53 / 768    0.258
54 / 768    0.938
55 / 768    0.507
56 / 768    0.674
58 / 768    0.73
59 / 768    0.178
61 / 768    0.534
62 / 768    0.364
63 / 768    0.78
64 / 768    0.103
66 / 768    0.155
67 / 768    0.571
68 / 768    0.371
69 / 768    0.2
70 / 7

In [6]:
from datetime import date

identifiers = ['W', 'H', 'P']

model_directory = str("./wflow_models/")
intbl_directory = str("./wflow_models/" + "/intbl/")

for n in wsName_list:
    static_directory = str("./wflow_models/" + n + "/staticmaps/")
    temp_directory = str("./wflow_models/" + n + "/temp/")

    #csv_file_path = os.path.join(str("./wflow_models/combined_samples_0716" + ".csv")) ##set this correctly!!!
    #csv_file_path = os.path.join(str("./wflow_models/combined_samples_0716_" + str(n) + ".csv")) ##set this correctly!!!
    #csv_file_path = os.path.join(str("./wflow_models/combined_samples_90_" + str(n) + ".csv")) ##set this correctly!!!
    csv_file_path = os.path.join(str("./wflow_models/san-diego_california_20331196_11023340_768.csv")) ##set this correctly!!!

    df = pd.read_csv(csv_file_path)
    
    if 'row_number_full' in df.columns:
        df['row_number_full'] = df['row_number_full'] - 1
        df.set_index('row_number_full', inplace=True)
    else:
        print("'row_number_full' column not found. Using default index.")

    print(str(os.path.abspath(temp_directory) +": processing"))
    
    for index, row in df.iterrows():
        
        print(f"\r{index + 1} / {len(df)}", end='    ', flush=True)
        temp_iter_directory = str(temp_directory+str(index))
        temp_iter_intbl_directory = temp_iter_directory + "/intbl/"
        
        #if not os.path.exists(temp_iter_directory):
        #    os.mkdir(temp_iter_directory)
            
        try:
            shutil.copytree(intbl_directory, temp_iter_intbl_directory, dirs_exist_ok=True)
        except Exception as e:
            print(f"Error occurred: {e}")
            
        TempBuilder.modify_and_save_ini2(model_directory+"/wflow_tofuflex.ini", index, [row.iloc[12],row.iloc[13], row.iloc[14], row.iloc[15]], temp_iter_directory+"/wflow_tofuflex_"+str(index)+".ini")
        TempBuilder.modify_and_save_ini2(model_directory+"/wflow_tofuflex_1km.ini", index, [row.iloc[12],row.iloc[13], row.iloc[14], row.iloc[15]], temp_iter_directory+"/wflow_tofuflex_1km_"+str(index)+".ini")
        TempBuilder.modify_and_save_ini2(model_directory+"/wflow_tofuflex_ns.ini", index, [row.iloc[12],row.iloc[13], row.iloc[14], row.iloc[15]], temp_iter_directory+"/wflow_tofuflex_ns_"+str(index)+".ini")
        TempBuilder.modify_and_save_ini2(model_directory+"/wflow_tofuflex_ns_1km.ini", index, [row.iloc[12],row.iloc[13], row.iloc[14], row.iloc[15]], temp_iter_directory+"/wflow_tofuflex_ns_1km_"+str(index)+".ini")
        TempBuilder.modify_and_save_ini2(model_directory+"/wflow_topoflex_bm.ini", index, [row.iloc[12],row.iloc[13], row.iloc[14], row.iloc[15]], temp_iter_directory+"/wflow_topoflex_bm_"+str(index)+".ini")
        TempBuilder.modify_and_save_ini2(model_directory+"/wflow_topoflex_bm_1km.ini", index, [row.iloc[12],row.iloc[13], row.iloc[14], row.iloc[15]], temp_iter_directory+"/wflow_topoflex_bm_1km_"+str(index)+".ini")
            
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/betaW"+".tbl", row.iloc[0], temp_iter_intbl_directory+"/betaW"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/betaAP"+".tbl", row.iloc[0], temp_iter_intbl_directory+"/betaAP"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/betaH"+".tbl", row.iloc[1], temp_iter_intbl_directory+"/betaH"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/betaP"+".tbl", row.iloc[2], temp_iter_intbl_directory+"/betaP"+".tbl")
        
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/DH"+".tbl", row.iloc[3], temp_iter_intbl_directory+"/DH"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/DP"+".tbl", row.iloc[4], temp_iter_intbl_directory+"/DP"+".tbl")

        # These three doesn't need to be separated by OG.
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/KfW"+".tbl", row.iloc[5], temp_iter_intbl_directory+"/KfW"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/KfW_OG"+".tbl", row.iloc[5], temp_iter_intbl_directory+"/KfW_OG"+".tbl") # It is identical to KfW, but a place holder for topo_bm to make life easier
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/KfaP"+".tbl", row.iloc[6], temp_iter_intbl_directory+"/KfaP"+".tbl")

        # Testing Kfimp with 1
        #TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/Kfimp"+".tbl", row.iloc[5], temp_iter_intbl_directory+"/Kfimp"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/Kfimp"+".tbl", 1.0, temp_iter_intbl_directory+"/Kfimp"+".tbl")

        # KfP_OG.tbl and KfH_OG.tbl is used for topo_bm, but process_Kf_files() will adjust it to KfP and KfH that goes into tofu model.
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/KfH"+".tbl", row.iloc[7], temp_iter_intbl_directory+"/KfH"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/KfH_OG"+".tbl", row.iloc[7], temp_iter_intbl_directory+"/KfH_OG"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/KfP"+".tbl", row.iloc[8], temp_iter_intbl_directory+"/KfP"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/KfP_OG"+".tbl", row.iloc[8], temp_iter_intbl_directory+"/KfP_OG"+".tbl")


        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/sumaxW"+".tbl", row.iloc[9], temp_iter_intbl_directory+"/sumaxW"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/sumaxH"+".tbl", row.iloc[10], temp_iter_intbl_directory+"/sumaxH"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/sumaxP"+".tbl", row.iloc[11], temp_iter_intbl_directory+"/sumaxP"+".tbl")
        
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/capW"+".tbl", row.iloc[16], temp_iter_intbl_directory+"/capW"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/percH"+".tbl", row.iloc[17], temp_iter_intbl_directory+"/percH"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/percP"+".tbl", row.iloc[17], temp_iter_intbl_directory+"/percP"+".tbl")

        if n == "san-diego_california_20331196_11023340":
            KsVal = 0.0002
        elif n == "fulton_georgia_2047963_02336410":
            KsVal = 0.0008
        
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/KsW"+".tbl", KsVal, temp_iter_intbl_directory+"/KsW"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/KsH"+".tbl", KsVal, temp_iter_intbl_directory+"/KsH"+".tbl")
        TempBuilder.modify_and_save_intbl(temp_iter_intbl_directory+"/KsP"+".tbl", KsVal, temp_iter_intbl_directory+"/KsP"+".tbl")
        
        TempBuilder.process_Kf_files(temp_iter_intbl_directory, identifiers, static_directory)
    print()



D:\wflow_models\san-diego_california_20331196_11023340\temp: processing
1024 / 768    


# Just the ini files for quicker update

In [4]:
model_directory = str("./wflow_models/")

for n in wsName_list:
    static_directory = str("./wflow_models/" + n + "/staticmaps/")
    temp_directory = str("./wflow_models/" + n + "/temp/")
    csv_file_path = os.path.join(str("./wflow_models/combined_samples_0716_" + str(n) + ".csv")) ##set this correctly!!!
    df = pd.read_csv(csv_file_path)
    print(str(os.path.abspath(temp_directory) +": processing just .ini files"))


    for index, row in df.iterrows():
        
        print(f"\r{index + 1} / {len(df)}", end='    ', flush=True)
        temp_iter_directory = str(temp_directory+str(index))
    
        TempBuilder.modify_and_save_ini2(model_directory+"/wflow_tofuflex.ini", index, [row.iloc[12],row.iloc[13], row.iloc[14], row.iloc[15]], temp_iter_directory+"/wflow_tofuflex_"+str(index)+".ini")
        TempBuilder.modify_and_save_ini2(model_directory+"/wflow_tofuflex_1km.ini", index, [row.iloc[12],row.iloc[13], row.iloc[14], row.iloc[15]], temp_iter_directory+"/wflow_tofuflex_1km_"+str(index)+".ini")
        TempBuilder.modify_and_save_ini2(model_directory+"/wflow_tofuflex_ns.ini", index, [row.iloc[12],row.iloc[13], row.iloc[14], row.iloc[15]], temp_iter_directory+"/wflow_tofuflex_ns_"+str(index)+".ini")
        TempBuilder.modify_and_save_ini2(model_directory+"/wflow_tofuflex_ns_1km.ini", index, [row.iloc[12],row.iloc[13], row.iloc[14], row.iloc[15]], temp_iter_directory+"/wflow_tofuflex_ns_1km_"+str(index)+".ini")
        TempBuilder.modify_and_save_ini2(model_directory+"/wflow_topoflex_bm.ini", index, [row.iloc[12],row.iloc[13], row.iloc[14], row.iloc[15]], temp_iter_directory+"/wflow_topoflex_bm_"+str(index)+".ini")
        TempBuilder.modify_and_save_ini2(model_directory+"/wflow_topoflex_bm_1km.ini", index, [row.iloc[12],row.iloc[13], row.iloc[14], row.iloc[15]], temp_iter_directory+"/wflow_topoflex_bm_1km_"+str(index)+".ini")
        
    print()

D:\wflow_models\san-diego_california_20331196_11023340\temp: processing just .ini files
256 / 256    
D:\wflow_models\fulton_georgia_2047963_02336410\temp: processing just .ini files
256 / 256    
